#### Submitting Somatic SNV callers, Manta and CNVkit
##### App used - `dev-canine-workflow/kfdrc-production-somatic-wgs-variant-sv-wf`

1) Importing modules 

In [69]:
import sevenbridges as sbg
import os
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
api = sbg.Api(url='https://cavatica-api.sbgenomics.com/v2', token='<token_id>',
        error_handlers=[rate_limit_sleeper, maintenance_sleeper])



2) Defining task inputs used for all samples

In [79]:
project="d3b-bixu/dev-canine-workflow"
app="d3b-bixu/dev-canine-workflow/kfdrc-production-somatic-wgs-variant-sv-wf"

ref = api.files.query(project="d3b-bixu/dev-canine-workflow", names = ["Canis_familiaris.CanFam3.1.dna.toplevel.fa"])[0]
ref_dict = api.files.query(project="d3b-bixu/dev-canine-workflow", names = ["Canis_familiaris.CanFam3.1.dna.toplevel.dict"])[0]
cnvkit_annotation_file = api.files.query(project="d3b-bixu/dev-canine-workflow", names = ["CanFam3_refFlat.txt"])[0]
lancet_bed = api.files.query(project="d3b-bixu/dev-canine-workflow", names = ["Canis_familiaris.CanFam3.1.86.exome.bed"])[0]
mutect2_af_vcf = api.files.query(project="d3b-bixu/dev-canine-workflow", names = ["92indsAEDPCDTaimyr.biallelic.up.sort.vcf.gz"])[0]
snpeff_db = api.files.query(project="d3b-bixu/dev-canine-workflow", names = ["snpEff_v4_3_CanFam3.1.86.zip"])[0]
strelka2_bed = api.files.query(project="d3b-bixu/dev-canine-workflow", names = ["Canis_WGS_withoutcontigs_withoutchrY.bed.gz"])[0]
wgs_calling_interval = api.files.query(project="d3b-bixu/dev-canine-workflow", names = ["Canis_WGS_withoutcontigs_withoutchrY.bed"])[0]


3) Defining sample specific variables using manifest, create `dict` as input and submit task using dict as input

In [92]:

manifest = open("tumor_and_normal_manifest.csv", "r")
manifestlines = manifest.readlines()

for line in manifestlines[1:]:
    line=line.split()
    normalbam = api.files.query(project="d3b-bixu/dev-canine-workflow", names = [line[1]])[0]
    tumorbam = api.files.query(project="d3b-bixu/dev-canine-workflow", names = [line[5]])[0]
    dict = {}
    dict["cnvkit_annotation_file"] = cnvkit_annotation_file
    dict["indexed_reference_fasta"]  = ref
    dict["input_normal_aligned"] = normalbam
    dict["input_tumor_aligned"] = tumorbam
    dict["lancet_calling_interval_bed"] = lancet_bed
    dict["mutect2_af_only_gnomad_vcf"] = mutect2_af_vcf
    dict["mutect2_exac_common_vcf"] = mutect2_af_vcf
    dict["reference_dict"]  = ref_dict
    dict["snpeff_database"] = snpeff_db
    dict["strelka2_bed"] = strelka2_bed
    dict["wgs_calling_interval_list"] = wgs_calling_interval
    dict["cnvkit_sex"] = line[2]
    dict["cnvkit_wgs_mode"] = "Y"
    dict["exome_flag"] = "N"
    dict["input_normal_name"] = line[8]
    dict["input_tumor_name"] = line[9]
    dict["snpeff_genomeversion"] = "CanFam3.1.86"
    dict["lancet_padding"] = 300
    dict["lancet_window"] = 600
    dict["select_vars_mode"] = "gatk"
    dict["vardict_padding"] = 150
    name =  "Somatic_callers_manta_cnvkit_"+line[0]
    task = api.tasks.create(name=name, project=project, app=app, inputs=dict, run=False)
    task.inputs['output_basename'] = task.id
    task.save()